In [1]:
import pandas as pd
import csv
import sys
import StringIO
import os

prepath = "/Users/mattzhang/Dropbox/Projects/Data/Zillow/"
properties = pd.read_csv(prepath+"properties_2016.csv")

/Users/mattzhang/py2_kernel/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# sensible dataset names, from https://www.kaggle.com/philippsp/exploratory-analysis-zillow
# also, make text-based values numeric

properties = properties.rename(columns={
    'parcelid': 'id_parcel',
    'yearbuilt': 'build_year',
    'basementsqft': 'area_basement',
    'yardbuildingsqft17': 'area_patio',
    'yardbuildingsqft26': 'area_shed',
    'poolsizesum': 'area_pool',
    'lotsizesquarefeet': 'area_lot',
    'garagetotalsqft': 'area_garage',
    'finishedfloor1squarefeet': 'area_firstfloor_finished',
    'calculatedfinishedsquarefeet': 'area_total_calc',
    'finishedsquarefeet6': 'area_base',
    'finishedsquarefeet12': 'area_live_finished',
    'finishedsquarefeet13': 'area_liveperi_finished',
    'finishedsquarefeet15': 'area_total_finished',
    'finishedsquarefeet50': 'area_unknown',
    'unitcnt': 'num_unit',
    'numberofstories': 'num_story',
    'roomcnt': 'num_room',
    'bathroomcnt': 'num_bathroom',
    'bedroomcnt': 'num_bedroom',
    'calculatedbathnbr': 'num_bathroom_calc',
    'fullbathcnt': 'num_bath',
    'threequarterbathnbr': 'num_75_bath',
    'fireplacecnt': 'num_fireplace',
    'poolcnt': 'num_pool',
    'garagecarcnt': 'num_garage',
    'regionidcounty': 'region_county',
    'regionidcity': 'region_city',
    'regionidzip': 'region_zip',
    'regionidneighborhood': 'region_neighbor',
    'taxvaluedollarcnt': 'tax_total',
    'structuretaxvaluedollarcnt': 'tax_building',
    'landtaxvaluedollarcnt': 'tax_land',
    'taxamount': 'tax_property',
    'assessmentyear': 'tax_year',
    'taxdelinquencyflag': 'tax_delinquency',
    'taxdelinquencyyear': 'tax_delinquency_year',
    'propertyzoningdesc': 'zoning_property',
    'propertylandusetypeid': 'zoning_landuse',
    'propertycountylandusecode': 'zoning_landuse_county',
    'fireplaceflag': 'flag_fireplace',
    'hashottuborspa': 'flag_tub',
    'buildingqualitytypeid': 'quality',
    'buildingclasstypeid': 'framing',
    'typeconstructiontypeid': 'material',
    'decktypeid': 'deck',
    'storytypeid': 'story',
    'heatingorsystemtypeid': 'heating',
    'airconditioningtypeid': 'aircon',
    'architecturalstyletypeid': 'architectural_style'
});

# treat no info as equal to no tax delinquency
properties['tax_delinquency'] = [int(i == 'Y') for i in properties['tax_delinquency']]
# assume no fireplace if not stated
properties['flag_fireplace'] = [int(i == 'Y') for i in properties['flag_fireplace']]
# assume no hot tub present
properties['flag_tub'] = [int(i == 'Y') for i in properties['flag_tub']]

In [3]:
errors = pd.read_csv(prepath+"train_2016_v2.csv")

In [4]:
errors = errors.assign(transaction_year = pd.Series([int(i.split('-')[0]) for i in errors['transactiondate']]).values)
errors = errors.assign(transaction_month = pd.Series([int(i.split('-')[1]) for i in errors['transactiondate']]).values)
errors = errors.assign(transaction_day = pd.Series([int(i.split('-')[2]) for i in errors['transactiondate']]).values)
errors = errors.drop('transactiondate', 1)

In [5]:
errors = errors.rename(columns={
    'parcelid': 'id_parcel'});

In [6]:
# stupid Zillow only gave us truth values for 90k events
goodIDs = errors['id_parcel'].values
properties = properties[properties['id_parcel'].isin(goodIDs)]

In [7]:
#duplicates = [[item]*count for item, count in Counter(errors['id_parcel'].values).iteritems() if count > 1]
#duplicates = np.array(duplicates).flatten()
#len(duplicates)
#duplicates[0]

In [8]:
# have to duplicate row when there's more than one sale for a single ID
from collections import Counter

duplicates = []
for item, count in Counter(errors['id_parcel'].values).iteritems():
    for _ in range(count-1):
        duplicates.append(item)

for i in duplicates:
    properties = properties.append(properties[properties.id_parcel == i].iloc[0])

In [9]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

Using TensorFlow backend.


In [10]:
X = properties.iloc[:,1:].values
y = errors[[1]].values.flatten()

In [12]:
# replaces all NaN with -1
#X = [[-1 if np.isnan(value) else value for value in row] for row in X]
#X = [[type(value) for value in row] for row in X]
#np.nan_to_num(X)
where_are_NaNs = np.isnan(X)
X[where_are_NaNs] = 0

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
X[0][0]

In [ ]:
print X[0][0] == float('nan')
print np.isnan(X[0][0])

In [ ]:
# define base model for scikit_learn wrapper
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=len(X[0]), kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=5, verbose=0)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))